## Antibody split

In [1]:
import sys
from sys import stdout
import inspect
from functional import seq
from pathlib import Path
from pycomfort.files import *
import biopandas
from biopandas.pdb import PandasPdb

In [2]:
debug_local = True#False
local = Path("..").resolve()
code = local / "mm"
data  = local / "data"
input = data / "input"
output = Path("/media/antonkulaga/Elements/molecular_dynamics")

if debug_local and code.exists():
    sys.path.insert(0, code.as_posix())
    print("extending pathes with local mm")
    print(sys.path)
    %load_ext autoreload
    %autoreload 2

extending pathes with local mm
['/data/sources/antibody-mm/mm', '/data/sources/antibody-mm/notebooks', '/home/antonkulaga/micromamba/envs/antibody-mm/lib/python39.zip', '/home/antonkulaga/micromamba/envs/antibody-mm/lib/python3.9', '/home/antonkulaga/micromamba/envs/antibody-mm/lib/python3.9/lib-dynload', '', '/home/antonkulaga/micromamba/envs/antibody-mm/lib/python3.9/site-packages', '/home/antonkulaga/micromamba/envs/antibody-mm/lib/python3.9/site-packages/IPython/extensions', '/home/antonkulaga/.ipython']


In [8]:
best_docking_results = Path("/data/samples/docking/RA/results/best")

# best_p2_auto = best_docking_results / "S5205Nr1-P2_IgG1Fc_H_top_30_heavy_chains"
tprint(best_p2_auto)

In [9]:
from splitter import split_and_write

In [5]:
from splitter import *

In [13]:
p = best_docking_results / "S5205Nr1-P2_IgG1Fc_H_top_30_heavy_chains" / "FIXED_1_35744_H_1_energy_-254.21349.pdb"

In [17]:
# load antibody key dataframes
antibody = read_pdb(p) # ugly, TODO: fix
antibody_atoms = antibody.df["ATOM"][antibody.df["ATOM"]["chain_id"] =="B"]
antibody_ids = antibody_atoms["atom_number"].values
antibody_others = antibody.df["OTHERS"]
antibody_ids_concat = "|".join(antibody_ids.astype(str))

# load key antigen dataframes
antigen = read_pdb(p) # ugly, TODO: fix
antigen_atoms = antigen.df["ATOM"][antibody.df["ATOM"]["chain_id"] =="A"]
antigen_ids = antigen_atoms["atom_number"].values
antigen_others = antigen.df["OTHERS"]
antigen_ids_concat = "|".join(antigen_ids.astype(str))

In [19]:
antibody_others[~((antibody_others["record_name"] == "TER") & (antibody_others["entry"].str.contains("A")))]

,record_name,entry,line_idx
0,REMARK,1 PDBFIXER FROM: 1_35744_H_1_energy_-254.21...,0
1,REMARK,"1 CREATED WITH OPENMM 7.7, 2022-04-07",1
3,TER,4881 LEU B 443,4882
4,CONECT,1992 2969,4883
5,CONECT,2969 1992,4884
6,CONECT,3688 4589,4885
7,CONECT,4589 3688,4886
8,END,,4887


In [22]:
#fixing issues in antibody
antibody_others_good_ter = antibody_others[~((antibody_others["record_name"] == "TER") & (antibody_others["entry"].str.contains("A")))]
antibody_others_updated = antibody_others_good_ter[~((antibody_others_good_ter["record_name"] == "CONECT") & (antibody_others_good_ter["entry"].str.contains(antigen_ids_concat)))]


#fixing issues in antigen
antigen_others_good_ter = antigen_others[~((antigen_others["record_name"] == "TER") & (antigen_others["entry"].str.contains("B")))]
antigen_others_updated = antigen_others_good_ter[~((antigen_others_good_ter["record_name"] == "CONECT") & (antigen_others_good_ter["entry"].str.contains(antibody_ids_concat)))]


In [23]:
antigen_others_updated

,record_name,entry,line_idx
0,REMARK,1 PDBFIXER FROM: 1_35744_H_1_energy_-254.21...,0
1,REMARK,"1 CREATED WITH OPENMM 7.7, 2022-04-07",1
2,TER,1605 SER A 105,1606
8,END,,4887


In [25]:
import shutil

for c in files(best_p2_auto):
    d = c.parent / c.stem
    #if d.exists():
    #    shutil.rmtree(d)
    if not d.exists():
        print(f"splitting the model {c} into antigen")
        split_and_write(c)

splitting the model /data/samples/docking/RA/results/best/S5205Nr1-P2_IgG1Fc_H_top_30_heavy_chains/FIXED_2_27457_H_244_energy_-284.44533.pdb into antigen
writing antibody to /data/samples/docking/RA/results/best/S5205Nr1-P2_IgG1Fc_H_top_30_heavy_chains/FIXED_2_27457_H_244_energy_-284.44533/FIXED_2_27457_H_244_energy_-284.44533_antibody.pdb
writing antigen to /data/samples/docking/RA/results/best/S5205Nr1-P2_IgG1Fc_H_top_30_heavy_chains/FIXED_2_27457_H_244_energy_-284.44533/FIXED_2_27457_H_244_energy_-284.44533_antigen.pdb
splitting the model /data/samples/docking/RA/results/best/S5205Nr1-P2_IgG1Fc_H_top_30_heavy_chains/3_17806_H_5_energy_-238.0523.pdb into antigen
writing antibody to /data/samples/docking/RA/results/best/S5205Nr1-P2_IgG1Fc_H_top_30_heavy_chains/3_17806_H_5_energy_-238.0523/3_17806_H_5_energy_-238.0523_antibody.pdb
writing antigen to /data/samples/docking/RA/results/best/S5205Nr1-P2_IgG1Fc_H_top_30_heavy_chains/3_17806_H_5_energy_-238.0523/3_17806_H_5_energy_-238.0523_a